<a href="https://colab.research.google.com/github/Sharyu-patil/Movie-recommendation/blob/master/Movie%20recommendation%20Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

movie_data = pd.read_csv('movies_metadata.csv', encoding='latin1', sep=',', on_bad_lines='skip', engine='python')
print(movie_data)
movie_data.head(2)
data_temp = movie_data["genres"]
movie_data['genres'] = movie_data['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
vote_counts = movie_data[movie_data['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = movie_data[movie_data['vote_average'].notnull()]['vote_average'].astype('int')
mean_vote_averages = vote_averages.mean()
mean_vote_averages
minimum_votes = vote_counts.quantile(0.95)
minimum_votes
md = movie_data
movie_data['year'] = pd.to_datetime(movie_data['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)


movie_data.columns
len(movie_data.columns)
qualified = md[(md['vote_count'] >= minimum_votes) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified
def Weighted_matrix(x):
    v = x['vote_count']
    r = x['vote_average']
    ans = (((v)/(v+minimum_votes))*r)+(((minimum_votes/(v+minimum_votes))*mean_vote_averages))
    return ans
qualified["weighted_rating"] = qualified.apply(Weighted_matrix,axis=1)
qualified = qualified.sort_values("weighted_rating",ascending=False)
qualified.head(10)
temp = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
temp.name = 'genre'
mgen_df = md.drop('genres', axis=1).join(temp)
def make_toplist(genre, percentile=0.85):
    df = mgen_df[mgen_df['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)

    col_list = ['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genre']
    qualified = df[(df['vote_count'] >= m)
                   & (df['vote_count'].notnull())
                   & (df['vote_average'].notnull())][col_list]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')

    qualified['weighted_rating'] = qualified.apply(lambda x:
                                                   (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C),
                                                   axis=1)
    qualified = qualified.sort_values('weighted_rating', ascending=False).head(250)

    return qualified
make_toplist('Crime').head(10)
movie_data.columns
movie_data[movie_data['adult']=='True']
small_mdf = pd.read_csv('links_small.csv')
small_mdf = small_mdf[small_mdf['tmdbId'].notnull()]['tmdbId'].astype('int')

def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan
md['id'] = md['id'].apply(convert_int)
md[md['id'].isnull()]
# md = md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int')
sm_df = md[md['id'].isin(small_mdf)]
sm_df['tagline'] = sm_df['tagline'].fillna('')
sm_df['description'] = sm_df['overview'] + sm_df['tagline']
sm_df['description'] = sm_df['description'].fillna('')
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=1, stop_words='english') # Changed min_df to 1
tfidf_matrix = tf.fit_transform(sm_df['description'])
# tfidf_matrix.shape
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
sm_df = sm_df.reset_index()
titles = sm_df['title']
indices = pd.Series(sm_df.index, index=sm_df['title'])
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:51]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]
inpt = input()
try:
    print(get_recommendations(inpt).head(10))
except:
    print("Movie not in the list")

#Toy Story
#Rebel Without a Cause
#Touch of Evil
#Laura

      adult                              belongs_to_collection    budget  \
0     False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1     False                                                NaN  65000000   
2     False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3     False                                                NaN  16000000   
4     False  {'id': 96871, 'name': 'Father of the Bride Col...         0   
...     ...                                                ...       ...   
3807  False                                                NaN         0   
3808  False  {'id': 259401, 'name': 'The Invisible Man (Uni...    328000   
3809  False                                                NaN         0   
3810  False                                                NaN         0   
3811  False                                                NaN   1500000   

                                                 genres  \
0     [{'id': 16, 'name': 'A

<ipython-input-1-6e5c511b75ec>:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sm_df['tagline'] = sm_df['tagline'].fillna('')
<ipython-input-1-6e5c511b75ec>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sm_df['description'] = sm_df['overview'] + sm_df['tagline']
<ipython-input-1-6e5c511b75ec>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Toy Story
2502                    Toy Story 2
2547                Man on the Moon
889           Rebel Without a Cause
1599                      Condorman
402               For Love or Money
437                          Malice
994                       Manhattan
1938                        Stepmom
3097    The Ballad of Ramblin' Jack
1792              Indecent Proposal
Name: title, dtype: object
